# Data set-up and cleaning

In [1]:
import pandas as pd
import numpy as np
from pandas import Series, DataFrame
import matplotlib.pyplot as plt
import psycopg2 as pg
%matplotlib inline
import seaborn as sns
import dbcreds

In [2]:
conn = pg.connect(dbname = dbcreds.database, host=dbcreds.host, user=dbcreds.user, password = dbcreds.password)

In [3]:
class color:
   PURPLE = '\033[95m'
   CYAN = '\033[96m'
   DARKCYAN = '\033[36m'
   BLUE = '\033[94m'
   GREEN = '\033[92m'
   YELLOW = '\033[93m'
   RED = '\033[91m'
   BOLD = '\033[1m'
   UNDERLINE = '\033[4m'
   END = '\033[0m'

In [4]:
apps_df = pd.read_sql('select * from cascais_v2.application', con=conn)


In [5]:
# create datetime objects for date of application
from datetime import datetime, timedelta
apps_df.loc[:,'app_date']= apps_df.loc[:,'candidatura_data'].apply(
    lambda x: datetime(year=int(str(x)[0:4]), month=int(str(x)[5:7]),
                       day=int(str(x)[8:10])))

# create english education levels
eng_edu = {'11': '11', '12': '12', '04': '04', '06': '06', '09':'09',
          'BM': 'bachelor', 'DT': 'doctoral', 'PS': 'post-secondary',
          'SL': 'grade school', 'LC': 'licensed',
          'MT': 'masters', 'NS': 'cannot read/write'}
    
apps_df.loc[:, 'education'] = apps_df['chabilitacao_escolar'].map(eng_edu)

# create english categories
eng_category = {'DESEMPREGADO-NOVO EMPREGO': 'Unemployed - new job',
          'DESEMPREGADO-1Âº EMPREGO': 'Unemployed - first job',
          'EMPREGADO': 'Employed',
          'EMPREGADO A TEMPO PARCIAL':'Part-time employed'}
apps_df.loc[:, 'category'] = apps_df['dcategoria'].map(eng_category)

# create english nationality
eng_nat = {'PORTUGAL':'PORTUGAL','BRASIL': 'BRAZIL',
           'GUINÃ-BISSAU':'GUINEA-BISSAU','CABO VERDE': 'CAPE VERDE'}

apps_df.loc[:,'nationality'] = apps_df['dnacionalidade'].apply(
lambda x: eng_nat[x] if x in eng_nat.keys() else 'OTHER')

#convert dependents to integer
apps_df.loc[:,'all_dependents'] = apps_df.loc[:,'ute_nr_pessoas_cargo'].apply(lambda x: None if x is None else float(x))
apps_df.loc[:,'descendants'] = apps_df.loc[:,'ute_nr_descendentes_cargo'].apply(lambda x: None if x is None else float(x))

In [6]:
#create age buckets
def age_bucket(x):
    if x<30:
        return "<30"
    elif x<50:
        return "30-49"
    elif x<65:
        return "50-64"
    else:
        return "65+"

apps_df.loc[:, 'age_bucket'] = apps_df.loc[:, 'ute_idade'].apply(lambda x: age_bucket(x))

In [7]:
#create # of apps
app_counts = apps_df.ute_id.value_counts()
apps_df['app_count'] = apps_df['ute_id'].apply(lambda x: app_counts[x])

In [8]:
# filter for only unemployed
apps_unemp = apps_df[apps_df['dcategoria'].apply(lambda x: x[0:3]==('DES'))]
len(apps_unemp)

119926

In [9]:
apps_unemp.columns

Index([u'anomes', u'ctipo_movimento', u'dtipo_movimento', u'ute_id', u'sexo',
       u'chabilitacao_escolar', u'dhabilitacao_escolar', u'cdeficiencia',
       u'ddeficiencia', u'cnacionalidade', u'dnacionalidade',
       u'cnp_pretendida', u'dcnp_pretendida', u'cpp_pretendida',
       u'dcpp_pretendida', u'candidatura_data', u'ccategoria', u'dcategoria',
       u'candidatura_motivo_inscricao', u'candidatura_rinsc',
       u'ute_estado_civil', u'candidatura_int_tempo_nao_insc',
       u'candidatura_prof_pret_tempo_pratica',
       u'sit_anterior_prof_tempo_pratica', u'candidatura_local_trabalho',
       u'ute_plano_emprego', u'ute_plano_emprego_anterior',
       u'candidatura_carteira_prof', u'candidatura_formacao_prof',
       u'candidatura_qualificacao', u'reinscricao_ult_saida_data',
       u'reinscricao_ult_saida_motivo', u'carea_formacao_tabela_em_activo',
       u'darea_formacao_tabela_em_activo', u'carea_curso_tabela_em_activo',
       u'darea_curso_tabela_em_activo', u'colocacao

## KY sanity checks

10. Understand data cleaning needed for "transfer of center" cancellation reasons

In [10]:
move_df = pd.read_sql('select * from cascais_v2.movement', con=conn)

In [11]:
move_df[move_df['movement_type']=='application'].groupby(['ute_id', 'movement_index']).count().head()

movement_event_date  application_id  movement_type  \
ute_id movement_index                                                       
710    120687                            1               1              1   
818    70870                             1               1              1   
820    22642                             1               1              1   
       61038                             1               1              1   
832    1941                              1               1              1   

                       movement_subtype  movement_result  
ute_id movement_index                                     
710    120687                         1                1  
818    70870                          1                1  
820    22642                          1                1  
       61038                          1                1  
832    1941                           1                1

In [12]:
apps_df.groupby(['ute_id', 'table_index']).count().head()

anomes  ctipo_movimento  dtipo_movimento  sexo  \
ute_id table_index                                                   
710    120687            1                1                1     1   
818    70870             1                1                1     1   
820    22642             1                1                1     1   
       61038             1                1                1     1   
832    1941              1                1                1     1   

                    chabilitacao_escolar  dhabilitacao_escolar  cdeficiencia  \
ute_id table_index                                                             
710    120687                          1                     1             1   
818    70870                           1                     1             1   
820    22642                           1                     1             1   
       61038                           1                     1             1   
832    1941                            1                     1             1   

                    ddeficiencia  cnacionalidade  dnacionalidade    ...      \
ute_id table_index                                                  ...       
710    120687                  1               1               1    ...       
818    70870                   1               1               1    ...       
820    22642                   1               1               1    ...       
       61038                   1               1               1    ...       
832    1941                    1               1               1    ...       

                    ute_nr_descendentes_cargo  candidatura_data_ppe  app_date  \
ute_id table_index                                                              
710    120687                               1                     1         1   
818    70870                                1                     0         1   
820    22642                                0                     0         1   
       61038                                1                     0         1   
832    1941                                 0                     0         1   

                    education  category  nationality  all_dependents  \
ute_id table_index                                                     
710    120687               1         1            1               1   
818    70870                1         1            1               1   
820    22642                1         1            1               1   
       61038                1         1            1               1   
832    1941                 1         1            1               1   

                    descendants  age_bucket  app_count  
ute_id table_index                                      
710    120687                 1           1          1  
818    70870                  1           1          1  
820    22642                  0           1          1  
       61038                  1           1          1  
832    1941                   0           1          1  

[5 rows x 67 columns]

In [13]:
merged = pd.merge(move_df, apps_df, how='inner', left_on='application_id', right_on='table_index')
#merged[['ute_id_x', 'ute_id_y']]
merged.loc[:, 'different']=merged['ute_id_x']!=merged['ute_id_y']

In [14]:
merged.different.sum()

0

In [15]:
from utils import db_utils
from utils import iefp_data_utils

ImportError: No module named utils

In [ ]:
clean_apps = iefp_data_utils.clean_applications(apps_df,'1980-01-01','2016-04-30')
clean_movs2 = iefp_data_utils.clean_movements(move_df, '1980-01-01','2017-04-30',clean_apps['table_index'])

In [ ]:
merged_clean = pd.merge(clean_movs2, clean_apps, how='inner', left_on='application_id', right_on='table_index')
merged_clean.loc[:, 'different']=merged['ute_id_x']!=merged['ute_id_y']

In [ ]:
merged_clean.different.sum()

In [ ]:
#count blank movement event dates
#move_df[move_df['movement_event_date']==''].groupby('movement_type').count()

In [ ]:
transfers = move_df[move_df['movement_subtype']=='TRANSFERÃNCIA DE CENTRO DE EMPREGO']

In [ ]:
transfers.head()

In [ ]:
plt.figure()
transfers.groupby('year').ute_id.count().plot(kind='bar')
plt.title("\nNumber of cancellations for Transfer de centro de emprego by year")

In [ ]:
#look at specific users' paths
move_df[move_df['ute_id']==97213]

In [ ]:
def next_move()

In [ ]:
move_df.head()

## Null analysis

In [ ]:
pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_rows', 1000)

def nulls_by_year(df):
    total_records = len(df)
    year_series = df.loc[:, 'anomes'].apply(lambda x: int(str(x)[0:4]))
    df_new = df.isnull().astype(int)
    df_new['year']=year_series
    
    df_new = df_new.groupby('year').sum()
    nulls_total = df_new.sum()
    nulls_total.name = 'nulls_total'
    
    return df_new.append(nulls_total).sort_values(by='nulls_total', ascending =False,axis=1).transpose().apply(lambda x: x/total_records*100)

nulls_by_year(apps_df)

## Create histograms (all apps))

In [ ]:
# plot number of apps by sex and category
status_order = ['Unemployed - new job', 'Unemployed - first job',
                'Part-time employed', 'Employed']

plt.figure()
sns.countplot(x='category', hue='sexo', data=apps_df, order=status_order)
plt.title("\n Number of applications by category and sex")
plt.xticks(rotation=45)

#plot dist of category by nationality
plt.figure()
sns.countplot(x='category', hue='nationality', data=apps_df, order=status_order)
plt.title("\nNumber of applications by category and nationality")
plt.xticks(rotation=45)

# plot number of times one applies (assumes sex and app count stays the same)
plt.figure()
first_app = apps_df.groupby('ute_id').first()
sns.countplot(x='app_count', hue='sexo', data=first_app)
plt.title("\n Number of users who applied X number of times between 2007-2017")



## Create histograms (unemployed only)

In [ ]:
#apps_unemp.columns
#apps_unemp.dnacionalidade.value_counts()

In [ ]:
#plot dist of ages by sex
plt.figure()
plt.hist(apps_unemp[apps_unemp['sexo']=='M'].ute_idade, bins=np.arange(15,80,1), color='indianred',
        alpha=0.5)
plt.hist(apps_unemp[apps_unemp['sexo']=='F'].ute_idade, bins=np.arange(15,80,1), alpha =0.4)
plt.title("\nDistribution of unemployed applications by age and sexo")

#plot dist of ages by sex
plt.figure()
sns.countplot(x='age_bucket', hue='sexo', data=apps_unemp)
plt.title("\nDistribution of unemployed applications by age bucket and sexo")



In [ ]:
edu_order = ['cannot read/write', 'grade school', 
            '04', '06', '09', '11', '12', 'post-secondary', 'licensed',
            'bachelor', 'masters', 'doctoral']
sns.countplot(x='education', hue='sexo', order=edu_order, data=apps_unemp)
plt.xticks(rotation=45)
plt.title("\nDistribution of applications by education and sexo (all ages)")

plt.figure()
sns.countplot(x='education', hue='sexo', order=edu_order,
              data=apps_unemp[apps_unemp['age_bucket']=="<30"])
plt.xticks(rotation=45)
plt.title("\nDistribution of applications by education and sexo (<30 y.o.)")

In [ ]:
edu_age = DataFrame(apps_unemp.groupby(['education', 'age_bucket']).ute_id.count())
edu_age = edu_age.reset_index().pivot( "age_bucket", "education","ute_id")
edu_age = edu_age.reindex(index = ["65+", "50-64","30-49","<30"], columns = edu_order)
edu_age = edu_age.fillna(value = 0)

fig, ax = plt.subplots(figsize = (10,3))
sns.heatmap(edu_age, annot=True, fmt='g', ax=ax)
plt.title("\nNumber of applications by education and age")

In [ ]:
plt.figure()
sns.countplot(x='sexo', hue='ute_estado_civil', data=apps_unemp)
plt.title("\nDistribution of applications by marital status and sexo")

plt.figure()
sns.countplot(x='age_bucket', hue='ute_estado_civil', data=apps_unemp)
plt.title("\nDistribution of applications by marital status and age")

In [ ]:
plt.figure()
sns.barplot(x="ute_estado_civil", y="all_dependents", hue='sexo',
            data=apps_unemp)
plt.title("\nAverage number of dependents by sex and marital status")

plt.figure()
sns.regplot(x="all_dependents",y="descendants", data=apps_unemp)



In [ ]:
des_dep = DataFrame(apps_unemp.groupby(['all_dependents', 'descendants']).ute_id.count())
des_dep = des_dep.reset_index().pivot( "all_dependents", "descendants","ute_id").sort_index(ascending=False)
#edu_age = edu_age.reindex(index = ["65+", "50-64","30-49","<30"], columns = edu_order)
#edu_age = edu_age.fillna(value = 0)

fig, ax = plt.subplots(figsize = (10,5))
sns.heatmap(des_dep, annot=True, fmt='g', ax=ax)
plt.title("\nNumber of dependents vs. Number of toal descendants")

## Deep dive on young people (<30 y.o.)

In [ ]:
events_df = pd.read_sql('select * from cascais.movement', con=conn)

In [ ]:
events_df.columns

In [ ]:
events_df.movement_index.sort_values(ascending=False)